In [21]:
# Import necessary modules for defining the neural network architecture and data transformations
import torch.nn as nn
from torchvision.transforms import transforms
import torch

# Import the pathlib module for working with file paths
import pathlib

In [22]:
# Define the root path where the training dataset is located
root = pathlib.Path("Dataset/Train")

# Extract and sort the class or category names based on subdirectories within the root directory
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

# Print the sorted list of class or category names
print(classes)

['None', 'Paper', 'Rock', 'Scissor']


In [23]:
# Define a series of transformations for converting images to tensors
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.Resize((320, 320)),  # Resize the image to 320x320 pixels
    transforms.ToTensor(),          # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

# Define a series of transformations for converting tensors to PIL images
transform_PIL = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),
    transforms.ToPILImage(),        # Convert the tensor to a PIL image
    transforms.Resize((320, 320)),  # Resize the PIL image to 320x320 pixels
    transforms.ToTensor(),          # Convert the PIL image back to a tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the tensor
])

In [24]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=len(classes)):
        super(ConvNet, self).__init__()
        
        # Define the first convolutional layer
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        
        # Define the second convolutional layer
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(num_features=24)
        self.relu2 = nn.ReLU()
        
        # Max-pooling layer after the second convolutional layer
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        
        # Define the third convolutional layer
        self.conv3 = nn.Conv2d(in_channels=24, out_channels=36, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=36)
        self.relu3 = nn.ReLU()
        
        # Define the fourth convolutional layer
        self.conv4 = nn.Conv2d(in_channels=36, out_channels=48, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(num_features=48)
        self.relu4 = nn.ReLU()
        
        # Max-pooling layer after the fourth convolutional layer
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected (dense) layer for classification
        self.fc1 = nn.Linear(in_features=48 * 80 * 80, out_features=256)
        self.fc2 = nn.Linear(in_features=256, out_features=num_classes)
           
    def forward(self, input):
        output = self.conv1(input)
        output = self.bn1(output)
        output = self.relu1(output)
        
        output = self.conv2(output)
        output = self.bn2(output)
        output = self.relu2(output)
        
        output = self.pool1(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)
        
        output = self.conv4(output)
        output = self.bn4(output)
        output = self.relu4(output)
        
        output = self.pool2(output)
        
        output = output.view(-1, 48 * 80 * 80)
        
        output = self.fc1(output)
        output = self.fc2(output)
        
        # Apply sigmoid activation to the output of the last linear layer
        output = torch.sigmoid(output)

        return output


In [25]:
# Create an instance of the ConvNet model with the number of output classes
model = ConvNet(num_classes=len(classes))